# Stokes 2D

Staggered-grid finite-difference implementation of the _Data-Assimilation Example from Glaciology_ given in [1].

In [1]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [2]:
# Parameters
alpha = 0.1/180*np.pi # inclination of the plane
height = 1.0e3        # height of the ice sheet (m)
length = 2.0e4        # length of the domain (m)
nx = 32               # number of grid points in x-direction
ny = 16               # number of grid points in y-direction
rho = 910.0           # density of ice, kg m^-3
mu = 1.0              # viscosity (temp)
g = 9.81              # gravitational acceleration (m s^-2)
A_glen = 1.0e-16      # Glen flow parameter (Pa^-n a^-1)
n_glen = 3            # Glen exponent
N = 30                # number of coefficients in the trigonometric expansion of beta^2

In [3]:
# Grid setup
hx = length/(nx-1)               # grid spacing in x-direction
hy = height/(ny-1)               # grid spacing in y-direction
x = np.arange(0,length+hx,hx)    # Note: +hx
y = np.arange(0,height+hy,hy)
X, Y = np.meshgrid(x,y)
Ux = (X[1:]+X[:-1])/2
Uy = (Y[1:]+Y[:-1])/2
Vx = (X[:,1:]+X[:,:-1])/2
Vy = (Y[:,1:]+Y[:,:-1])/2
Px = (X[1:,1:]+X[:-1,:-1])/2
Py = (Y[1:,1:]+Y[:-1,:-1])/2

## References

[1] Granzow, G. (2014). A tutorial on adjoint methods and their use for data assimilation in glaciology. Journal of Glaciology, 60(221), 440-446. doi:10.3189/2014JoG13J205

[2] Becker, T. W. and B. J. P. Kaus (2013). Numerical Modeling of Earth Systems. An introduction to computational methods with focus on solid Earth applications of continuum mechanics. Lecture notes, available online at http://www-udc.ig.utexas.edu/external/becker/Geodynamics557.pdf, accessed 01/2018.